In [2]:
from tkinter.filedialog import askopenfilename
import soundfile as sf
import sounddevice as sd
import tkinter as tk
import tkinter.ttk as ttk

class Audio(tk.Variable):
    def __init__(self):
        self.audio = None
        self.fm = -1

    def get(self):
        return self.audio, self.fm

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm

def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')
    def selecAudio():
        fichero = askopenfilename(title='Selecciona un fichero de audio',
                                filetypes=(('Fichero de audio', '.mp3 .wav'),
                                            ('Todos los archivos', '*.*')),
                                )
        if fichero:
            varMensaje.set(fichero)
            senyal, fm = sf.read(fichero)
            varAudio.set(senyal, fm)

    boton = ttk.Button(raiz, textvariable=varMensaje, command=selecAudio)
    boton.pack()

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM)

    repro = ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen))
    repro.pack(side=tk.LEFT)

    parar = ttk.Button(marco, text='Stop', command=sd.stop)
    parar.pack(side=tk.LEFT)

    salir = ttk.Button(marco, text='Quit', command=raiz.destroy)
    salir.pack(side=tk.LEFT)

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    volumen = ttk.Scale(marco, from_=0, to=1, length=200, value=varVolumen.get(),
        variable=varVolumen)
    volumen.pack()

def main():
    win = tk.Tk()
    win.title('Reproductor Sencillo de Audio')
    win.geometry('600x200')
    win.configure(background='Lavender Blush')

    ttk.Style().configure('.', font=("Arial", 16), background='Lavender Blush', padding=8)

    varAudio = Audio()
    varVolumen = tk.DoubleVar(win, 0.6)

    selecAudio(win, varAudio)
    reproduceParaSal(win, varAudio, varVolumen)
    volumen(win, varVolumen)

    win.mainloop()

if __name__ == '__main__':
    main()

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfilename
from tkinter.filedialog import asksaveasfilename
import soundfile as sf
import sounddevice as sd

class Audio(tk.Variable):
    def __init__(self):
        self.audio = None
        self.fm = -1

    def get(self):
        return self.audio, self.fm

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm

def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')

    def abrirArchivo():
        fichero = askopenfilename(
            title='Selecciona un fichero de audio',
            filetypes=(('Fichero de audio', '.mp3 .wav'), ('Todos los archivos', '*.*'))
        )
        if fichero:
            varMensaje.set(fichero)
            senyal, fm = sf.read(fichero)
            varAudio.set(senyal, fm)

    boton = ttk.Button(raiz, textvariable=varMensaje, command=abrirArchivo)
    boton.pack()

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM)

    ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen)).pack(side=tk.LEFT)
    ttk.Button(marco, text='Stop', command=sd.stop).pack(side=tk.LEFT)
    ttk.Button(marco, text='Quit', command=raiz.quit).pack(side=tk.LEFT)

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    ttk.Scale(marco, from_=0, to=1, length=200, variable=varVolumen).pack()

def estereo2mono(varAudio, varOutput, canal=2):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # Confirm it's stereo
    if audio.ndim != 2 or audio.shape[1] != 2:
        raise ValueError("La señal de entrada no es estéreo.")

    L = audio[:, 0]
    R = audio[:, 1]

    if canal == 0:
        mono = L
    elif canal == 1:
        mono = R
    elif canal == 2:
        mono = (L + R) / 2
    elif canal == 3:
        mono = (L - R) / 2
    else:
        raise ValueError("Canal debe ser 0, 1, 2 o 3.")

    # Save mono as 2D array for sounddevice compatibility
    mono = mono.reshape(-1, 1)
    varOutput.set(mono, fm)


def mono2estereo(varAudio, varOutput):
    import numpy as np

    audio, fm = varAudio.get()
    if audio is None:
        return  # No input audio

    # If already stereo, just pass through or raise error
    if audio.ndim == 2 and audio.shape[1] == 2:
        raise ValueError("La señal ya es estéreo.")

    # Make sure audio is 1D for mono
    if audio.ndim == 2 and audio.shape[1] == 1:
        audio = audio[:, 0]

    # Duplicate mono to stereo by stacking channels
    stereo = np.column_stack((audio, audio))

    varOutput.set(stereo, fm)


def funModif_3(raiz, varAudio, varOutput):
    print("Función de modificación 3 ejecutada")
def funModif_4(raiz, varAudio, varOutput):
    print("Función de modificación 4 ejecutada")

# === New buttons ===
def botonExtra1(raiz, varAudio, varOutput, num):
    def repartir(varAudio, varOutput, num):
        if num == 1:
            estereo2mono(varAudio, varOutput)
        elif num == 2:
            mono2estereo(varAudio, varOutput)
        elif num == 3:
            funModif_3(varAudio, varOutput)
        else:
            funModif_4(varAudio, varOutput)

    titulos = {
        1: 'Filtrar Ruido',
        2: 'Normalizar Volumen',
        3: 'Invertir Señal',
        4: 'Aplicar Eco'
    }
    texto_boton = titulos[num]

    ttk.Button(
        raiz,
        text=texto_boton,
        command=lambda: repartir(varAudio, varOutput, num)
    ).pack(side=tk.LEFT, padx=10)



def botonExtra2(raiz, varOutput):
    def guardarAudio():
        audio, fm = varOutput.get()
        if audio is not None:
            archivo = asksaveasfilename(
                title='Guardar archivo de audio',
                defaultextension='.wav',
                filetypes=(('Archivo WAV', '*.wav'), ('Todos los archivos', '*.*'))
            )
            if archivo:
                sf.write(archivo, audio, fm)
                print(f'Audio guardado en: {archivo}')
        else:
            print("No hay audio para guardar.")

    ttk.Button(raiz, text='Guardar Audio', command=guardarAudio).pack(side=tk.LEFT)

# === Audio player layout per tab ===
def crearTabContenido(tab, num):
    varAudio = Audio()
    varOutput = Audio()
    varVolumen = tk.DoubleVar(tab, 0.6)

    selecAudio(tab, varAudio)

    # Contenedor para los botones extra
    marcoBotonesExtra = ttk.Frame(tab)
    marcoBotonesExtra.pack(pady=10)
    botonExtra1(marcoBotonesExtra, varAudio, varOutput, num)
    botonExtra2(marcoBotonesExtra, varOutput)

    volumen(tab, varVolumen)
    reproduceParaSal(tab, varAudio, varVolumen)

# === Main app ===
def main():
    root = tk.Tk()
    root.title("Notebook con Reproductores de Audio")
    root.geometry("600x300")
    root.configure(background='Lavender Blush')
    ttk.Style().configure('.', font=("Arial", 14), background='Lavender Blush', padding=8)

    notebook = ttk.Notebook(root)
    notebook.pack(expand=True, fill='both')

    for i in range(4):
        tab = ttk.Frame(notebook)
        notebook.add(tab, text=f'Tab {i+1}')
        crearTabContenido(tab, num=i + 1)

    root.mainloop()

if __name__ == '__main__':
    main()


In [11]:
import tkinter as tk
import tkinter.ttk as ttk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import numpy as np
import soundfile as sf
import sounddevice as sd
import struct

# --- Helper functions to read/write WAV headers for encoded audio ---

def write_encoded_wav(filename, data, samplerate):
    # data: numpy int32 array
    byte_data = struct.pack('<' + 'i' * len(data), *data)
    with open(filename, 'wb') as out:
        out.write(struct.pack('<4sI4s', b'RIFF', 36 + len(byte_data), b'WAVE'))
        # Format chunk: PCM(1), mono(1), 32 bits/sample
        out.write(struct.pack('<4sIHHIIHH', b'fmt ', 16, 1, 1, samplerate,
                              samplerate * 4, 4, 32))
        out.write(struct.pack('<4sI', b'data', len(byte_data)))
        out.write(byte_data)

def read_encoded_wav(filename):
    with open(filename, 'rb') as f:
        # Read header chunks (simplified)
        riff = f.read(12)
        fmt = f.read(24)
        data_header = f.read(8)
        # Extract sample rate and data size
        samplerate = struct.unpack('<I', fmt[12:16])[0]
        datasize = struct.unpack('<I', data_header[4:8])[0]
        data = f.read(datasize)
        samples = struct.unpack('<' + 'i' * (datasize // 4), data)
        return np.array(samples, dtype=np.int32), samplerate

# --- Audio class with audio + sample rate ---

class Audio(tk.Variable):
    def __init__(self):
        super().__init__()
        self.audio = None  # numpy array
        self.fm = -1      # sample rate

    def get(self):
        return self.audio, self.fm

    def set(self, audio, fm):
        self.audio = audio
        self.fm = fm

# --- File selection button ---

def selecAudio(raiz, varAudio):
    varMensaje = tk.StringVar(raiz, 'Selecciona una señal de audio')
    def selecAudio():
        fichero = askopenfilename(title='Selecciona un fichero de audio',
                                filetypes=(('WAV Files', '*.wav'),
                                           ('Todos los archivos', '*.*')))
        if fichero:
            varMensaje.set(fichero)
            data, samplerate = sf.read(fichero, dtype='float32')
            varAudio.set(data, samplerate)
    boton = ttk.Button(raiz, textvariable=varMensaje, command=selecAudio)
    boton.pack(pady=5)

# --- Playback and control buttons ---

def reproduce(varAudio, varVolumen):
    audio, fm = varAudio.get()
    if audio is not None:
        sd.play(audio * varVolumen.get(), fm, blocking=False)

def reproduceParaSal(raiz, varAudio, varVolumen):
    marco = ttk.Frame(raiz)
    marco.pack(side=tk.BOTTOM, pady=5)

    repro = ttk.Button(marco, text='Play', command=lambda: reproduce(varAudio, varVolumen))
    repro.pack(side=tk.LEFT, padx=5)

    parar = ttk.Button(marco, text='Stop', command=sd.stop)
    parar.pack(side=tk.LEFT, padx=5)

    salir = ttk.Button(marco, text='Quit', command=raiz.destroy)
    salir.pack(side=tk.LEFT, padx=5)

    return repro  # return Play button to enable/disable if needed

# --- Volume control ---

def volumen(raiz, varVolumen):
    marco = ttk.LabelFrame(raiz, text='Volumen')
    marco.pack()
    volumen = ttk.Scale(marco, from_=0, to=1, length=200, value=varVolumen.get(),
                       variable=varVolumen)
    volumen.pack()

# --- Extra buttons for each tab ---

def botonExtra1(raiz, varAudio, varOutput, num):
    def repartir():
        if num == 1:
            funModif_1(varAudio, varOutput)
        elif num == 2:
            funModif_2(varAudio, varOutput)
        elif num == 3:
            funModif_3(varAudio, varOutput)
        else:
            funModif_4(varAudio, varOutput)

    text_map = {
        1: 'Aplicar Mono',
        2: 'Aplicar Stereo',
        3: 'Codificar Stereo',
        4: 'Decodificar Stereo',
    }

    ttk.Button(
        raiz,
        text=text_map.get(num, 'Aplicar Modificación'),
        command=repartir
    ).pack(side=tk.LEFT, padx=10)

def botonExtra2(raiz, varOutput):
    def guardar():
        audio, fm = varOutput.get()
        if audio is None:
            return
        fichero = asksaveasfilename(defaultextension='.wav',
                                    filetypes=[('WAV files', '*.wav'), ('All files', '*.*')])
        if fichero:
            # Determine if data is encoded or normal by dtype and shape
            if audio.dtype == np.int32 and audio.ndim == 1:
                # encoded audio
                write_encoded_wav(fichero, audio, fm)
            else:
                # normal audio save
                sf.write(fichero, audio, fm)
    ttk.Button(
        raiz,
        text='Guardar Audio',
        command=guardar
    ).pack(side=tk.LEFT, padx=10)

# --- Audio modification functions ---

def funModif_1(varAudio, varOutput):
    # Stereo to Mono (simple average of channels)
    audio, fm = varAudio.get()
    if audio is None:
        return
    if audio.ndim == 1:
        # Already mono
        varOutput.set(audio, fm)
        return
    if audio.shape[1] == 2:
        mono = audio.mean(axis=1)
        varOutput.set(mono, fm)
    else:
        varOutput.set(audio, fm)

def funModif_2(varAudio, varOutput):
    # Mono to Stereo (duplicate mono channel)
    audio, fm = varAudio.get()
    if audio is None:
        return
    if audio.ndim == 1:
        stereo = np.column_stack((audio, audio))
        varOutput.set(stereo, fm)
    else:
        varOutput.set(audio, fm)

def funModif_3(varAudio, varOutput):
    # Encode stereo audio into int32 coded mono
    audio, fm = varAudio.get()
    if audio is None:
        return
    if audio.ndim != 2 or audio.shape[1] != 2:
        print("Error: input must be stereo audio.")
        return

    int_audio = (audio * 32767).astype(np.int16)
    L = int_audio[:, 0].astype(np.int32)
    R = int_audio[:, 1].astype(np.int32)
    codificats = ((L + R) << 16) | ((L - R) & 0xFFFF)
    varOutput.set(codificats, fm)

def funModif_4(varAudio, varOutput):
    # Decode int32 coded mono into stereo audio
    encoded, fm = varAudio.get()
    if encoded is None:
        return
    if not (encoded.dtype == np.int32 and encoded.ndim == 1):
        print("Error: input must be encoded int32 array.")
        return

    def signed_16bit(val):
        return val if val < 0x8000 else val - 0x10000

    vec_signed_16bit = np.vectorize(signed_16bit)
    lower_16 = encoded & 0xFFFF
    lower_16_signed = vec_signed_16bit(lower_16)
    upper_16 = encoded >> 16

    L = ((upper_16 + lower_16_signed) // 2).astype(np.int16)
    R = ((upper_16 - lower_16_signed) // 2).astype(np.int16)
    stereo = np.column_stack((L, R))
    stereo_float = stereo.astype(np.float32) / 32767.0
    varOutput.set(stereo_float, fm)

# --- Tab content creator ---

def crearTabContenido(tabControl, num):
    tab = ttk.Frame(tabControl)
    varAudio = Audio()
    varOutput = Audio()
    varVolumen = tk.DoubleVar(tabControl, 0.6)

    selecAudio(tab, varAudio)
    botonExtra1(tab, varAudio, varOutput, num)
    botonExtra2(tab, varOutput)
    vol_frame = ttk.Frame(tab)
    vol_frame.pack()
    volumen(vol_frame, varVolumen)
    play_btn = reproduceParaSal(tab, varOutput, varVolumen)

    # Disable play button for encoded audio (tab 3)
    if num == 3:
        play_btn.state(['disabled'])
    else:
        play_btn.state(['!disabled'])

    return tab

# --- Main ---

def main():
    win = tk.Tk()
    win.title('Reproductor audio de senyals - v4 Completo')

    tabControl = ttk.Notebook(win)
    tabs = []
    for i in range(1, 5):
        tab = crearTabContenido(tabControl, i)
        tabControl.add(tab, text=f'TAB {i}')
        tabs.append(tab)
    tabControl.pack(expand=1, fill='both')

    win.mainloop()

if __name__ == '__main__':
    main()
